## Import libraries 

In [1]:
import os
import pandas as pd
import pickle
import numpy
import numpy as np
import json

import matplotlib.pyplot as plt
%matplotlib inline

## Import Data 

In [2]:
dataset = pd.read_csv('../UDS.csv', low_memory=False)

In [3]:
dataset.head()

,NACCADC,NACCID,NACCVNUM,FORMVER,PACKET,VISITMO,VISITDAY,VISITYR,NACCREAS,NACCREFR,...,NACCDSMO,NACCDSDY,NACCDSYR,NACCNURP,NACCNRMO,NACCNRDY,NACCNRYR,ADGCGWAS,ADGCADSP,ADGCRND
0,1416,NACC000011,1,1,I,4,17,2006,1,8,...,3,1,2010,0,88,88,8888,0,0,88
1,1416,NACC000011,2,1,F,6,18,2007,1,8,...,3,1,2010,0,88,88,8888,0,0,88
2,1416,NACC000011,3,2,F,6,3,2008,1,8,...,3,1,2010,0,88,88,8888,0,0,88
3,1416,NACC000011,4,2,F,8,3,2009,1,8,...,3,1,2010,0,88,88,8888,0,0,88
4,9661,NACC000034,1,3,I,7,16,2015,1,1,...,88,88,8888,0,88,88,8888,0,0,88


In [4]:
data12 = dataset

In [5]:
len(data12[(data12['PACKET']=='I') & (data12['FORMVER'].isin([1,2])) & (data12['NACCUDSD']==4)])

12136

In [6]:
V12 = data12[(data12['PACKET']=='I') & (data12['FORMVER'].isin([1,2])) & (data12['NACCUDSD'].isin([1,2,3]))]

In [7]:
V12

,NACCADC,NACCID,NACCVNUM,FORMVER,PACKET,VISITMO,VISITDAY,VISITYR,NACCREAS,NACCREFR,...,NACCDSMO,NACCDSDY,NACCDSYR,NACCNURP,NACCNRMO,NACCNRDY,NACCNRYR,ADGCGWAS,ADGCADSP,ADGCRND
0,1416,NACC000011,1,1,I,4,17,2006,1,8,...,3,1,2010,0,88,88,8888,0,0,88
5,2096,NACC000067,1,2,I,5,23,2012,1,2,...,88,88,8888,0,88,88,8888,0,0,88
9,2289,NACC000144,1,2,I,8,17,2010,1,1,...,88,88,8888,0,88,88,8888,1,0,ADC 4
11,5452,NACC000162,1,2,I,3,10,2009,1,8,...,88,88,8888,0,88,88,8888,1,0,ADC 7
20,6518,NACC000225,1,2,I,9,23,2009,1,8,...,4,1,2016,0,88,88,8888,0,0,88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112690,5452,NACC999839,1,2,I,5,6,2008,1,8,...,88,88,8888,0,88,88,8888,1,0,ADC 4
112700,8361,NACC999854,1,1,I,8,14,2006,1,2,...,88,88,8888,0,88,88,8888,1,1,ADC 4
112712,8646,NACC999872,1,1,I,9,2,2005,1,1,...,88,88,8888,0,88,88,8888,1,0,ADC 3
112715,2578,NACC999922,1,2,I,8,7,2012,1,1,...,7,28,2014,0,88,88,8888,0,0,88


### Use patient id, 'NACCID', to find outcome at 1 and 2 years

In [8]:
def calculate_months(year1, month1, year, month):
    return (year-year1)*12 + (month-month1)

In [9]:
def find_cause(data, columns):
    
    loc = np.where(data==1)[0]
    
    if len(loc)==0:
        
        return -1
    
    else:
        
        return columns[loc[0]]    

In [10]:
diagnosis_fields = ['NACCALZD', 'NACCLBDE', 'VASC']

In [11]:
V12_id = V12['NACCID'].values

In [12]:
%%time

V12_outcome, V12_used, V12_months, V12_cause, V12_whodid = [],[],[],[],[]

count=0
count2=0

for patient in V12_id:
    
    outcomes = data12[(data12['NACCID']==patient) & (data12['NACCVNUM'].isin([2,3]))]
    
    if len(outcomes)==0:
        
        count+=1
        
        continue
    
    else:
        
        year0 = data12[(data12['NACCID']==patient) & (data12['NACCVNUM']==1)]['VISITYR'].values[0]
        month0 = data12[(data12['NACCID']==patient) & (data12['NACCVNUM']==1)]['VISITMO'].values[0]
        
            
        loc = 0
        value = outcomes['NACCUDSD'].values[0]
            
        year1 = outcomes['VISITYR'].values[loc]
        month1 = outcomes['VISITMO'].values[loc]
            
        months = calculate_months(year0, month0, year1, month1) 
            
        if months>29:
            
            count2+=1
                
            continue
                
        diagnosis = value
                
        if diagnosis==4:
            
            V12_outcome.append(1)
            V12_months.append(months)
            V12_used.append(patient)
            V12_cause.append(find_cause(outcomes[diagnosis_fields].values[loc],diagnosis_fields))
            V12_whodid.append(outcomes['WHODIDDX'].values[loc])
            
            continue
                    
        elif len(outcomes)>1:
                    
            year2 = outcomes['VISITYR'].values[loc+1]
            month2 = outcomes['VISITMO'].values[loc+1]

            months2 = calculate_months(year0, month0, year2, month2) 
            
            if months2<=29:
                    
                diagnosis = outcomes['NACCUDSD'].values[loc+1]
                        
                if diagnosis==4:
            
                    V12_outcome.append(1)
                    V12_months.append(months2)
                    V12_used.append(patient)
                    V12_cause.append(find_cause(outcomes[diagnosis_fields].values[loc+1],diagnosis_fields))
                    V12_whodid.append(outcomes['WHODIDDX'].values[loc+1])
                    continue
                        
                else:
                    V12_outcome.append(0)
                    V12_months.append(months2)
                    V12_used.append(patient)
                    V12_whodid.append(outcomes['WHODIDDX'].values[loc+1])
                    V12_cause.append(0)
                            
            else:

                V12_outcome.append(0)
                V12_months.append(months)
                V12_used.append(patient)
                V12_whodid.append(outcomes['WHODIDDX'].values[loc])
                V12_cause.append(0)
                        
        else:
                    
            V12_outcome.append(0)
            V12_months.append(months)
            V12_used.append(patient)
            V12_whodid.append(outcomes['WHODIDDX'].values[loc])
            V12_cause.append(0)
                        


            
        


CPU times: user 5min 46s, sys: 684 ms, total: 5min 46s
Wall time: 5min 47s


In [13]:
initial = data12[(data12['NACCVNUM']==1)]

len(initial[initial['NACCUDSD']==4])

12375

In [14]:
print('Number of patients with no follow-up: {0}'.format(count))

print('Number of patients with first follow-up outside of time window: {0}'.format(count2))

Number of patients with no follow-up: 4557
Number of patients with first follow-up outside of time window: 573


In [15]:
np.save('outcome_causes.npy', V12_cause)
np.save('who_diagnosed.npy', V12_whodid)

### Who (1 = single, 2=consensus) 

In [16]:
single = sum([1 for i,w in enumerate(V12_whodid) if w==1 and V12_outcome[i]==1])
consensus = sum([1 for i,w in enumerate(V12_whodid) if w==2 and V12_outcome[i]==1])

In [17]:
single, single/sum(V12_outcome)

(273, 0.17410714285714285)

In [18]:
consensus, consensus/sum(V12_outcome)

(1216, 0.7755102040816326)

In [19]:
sum([1 for i,w in enumerate(V12_whodid) if w==-4 and V12_outcome[i]==1])

79

### Combine outcomes with processed data 

In [20]:
V12_cleaned = pd.read_csv('UDSDataV12.csv', low_memory=False)
V12_cleaned = V12_cleaned[V12_cleaned['NACCID'].isin(V12_used)]

sum(V12_cleaned['NACCID'].values == V12_used), len(V12_outcome)

(15307, 15307)

In [21]:
V12_cleaned.drop(['NACCID', 'FORMVER', 'VISIT_DATE', 'NACCVNUM'], axis=1, inplace=True)

### Add 4 randomly distributed variables 

In [22]:
for i in range(1):
    
    var_name = f'RANDVAR_{i}'
    
    var_val = np.random.normal(np.random.randint(10), np.random.randint(1,5), len(V12_cleaned))
    
    V12_cleaned[var_name] = var_val
    
for i,var_shuffle in enumerate(['CDRSUM', 'INSEX', 'TRAILB_PROB']):
    
    var_name = f'RANDVAR_{i+1}'
    
    vals = V12_cleaned[var_shuffle].values
    
    var_val = np.random.permutation(vals)
    
    V12_cleaned[var_name] = var_val

## Population characteristics 

In [23]:
dementia_loc = np.array([i for i,v in enumerate(V12_outcome) if v==1])
nc_loc = np.array([i for i,v in enumerate(V12_outcome) if v==0])

#### Race

In [24]:
nodem = [v for i,v in enumerate(V12_used) if V12_outcome[i]==0]
dem = [v for i,v in enumerate(V12_used) if V12_outcome[i]==1]

original_nodem = data12[(data12['NACCID'].isin(nodem)) & (data12['NACCVNUM']==1)] 
original_dem = data12[(data12['NACCID'].isin(dem)) & (data12['NACCVNUM']==1)] 

In [25]:
non_white = [1 if v in [2,3,4,5,50] else 0 for v in original_dem['RACE'].values]

sum(non_white), sum(non_white)/len(non_white)

(221, 0.14094387755102042)

#### Independence 

In [26]:
dependant = [1 if v in [2,3,4] else 0 for v in original_nodem['INDEPEND'].values]

sum(dependant), sum(dependant)/len(dependant)

(927, 0.06747215954581848)

#### Age 

In [27]:
V12_cleaned['NACCAGE'].values.mean(), V12_cleaned['NACCAGE'].values.std(), 

(72.31965767296009, 9.82669755200655)

#### Gender 

In [28]:
sum([1 for v in V12_cleaned['SEX'].values[nc_loc] if v==1]), sum([1 for v in V12_cleaned['SEX'].values[nc_loc] if v==1])/len(nc_loc)

(5376, 0.39129485406507025)

#### Language 

In [29]:
sum([1 for v in V12_cleaned['PRIMLANG'].values[dementia_loc] if v==1]), sum([1 for v in V12_cleaned['PRIMLANG'].values[dementia_loc] if v==1])/len(dementia_loc)

(1471, 0.9381377551020408)

In [30]:
sum([1 for v in V12_cleaned['PRIMLANG'].values[nc_loc] if v==1]), sum([1 for v in V12_cleaned['PRIMLANG'].values[nc_loc] if v==1])/len(nc_loc)

(12823, 0.9333284809665915)

#### Education 

In [31]:
np.array([v for v in V12_cleaned['EDUC'].values[nc_loc] if not pd.isnull(v)]).mean(), np.array([v for v in V12_cleaned['EDUC'].values[nc_loc] if not pd.isnull(v)]).std()

(15.502813710443617, 3.186734380521302)

In [32]:
np.array([v for v in V12_cleaned['EDUC'].values[dementia_loc] if not pd.isnull(v)]).mean(), np.array([v for v in V12_cleaned['EDUC'].values[dementia_loc] if not pd.isnull(v)]).std()

(15.312020460358056, 3.3073325897037344)

#### CDR sum 

In [33]:
np.median([v for v in V12_cleaned['CDRSUM'].values[dementia_loc] if not pd.isnull(v)])

1.5

In [34]:
np.percentile([v for v in V12_cleaned['CDRSUM'].values[dementia_loc] if not pd.isnull(v)], [25,75])

array([1. , 2.5])

In [35]:
np.percentile([v for v in V12_cleaned['CDRSUM'].values[nc_loc] if not pd.isnull(v)], [25,75])

array([0. , 0.5])

#### DECCLIN 

In [36]:
sum([1 for v in V12_cleaned['DECCLIN'].values[nc_loc] if v==0])

9918

In [37]:
sum([1 for v in V12_cleaned['DECCLIN'].values[nc_loc] if v==0])/len(nc_loc)

0.7218866001892423

#### ANIMALS 

In [38]:
np.array([v for v in V12_cleaned['ANIMALS'].values[nc_loc] if not pd.isnull(v)]).mean()

19.18694204872144

In [39]:
 np.array([v for v in V12_cleaned['ANIMALS'].values[nc_loc] if not pd.isnull(v)]).std()

5.700691772464361

#### Trail B 

In [40]:
np.array([v for v in V12_cleaned['TRAILB'].values[dementia_loc] if not pd.isnull(v)]).mean()

160.7271418286537

In [41]:
 np.array([v for v in V12_cleaned['TRAILB'].values[dementia_loc] if not pd.isnull(v)]).std()

80.96926145617324

#### VEG 

In [42]:
np.array([v for v in V12_cleaned['VEG'].values[nc_loc] if not pd.isnull(v)]).mean()

13.837664322837053

In [43]:
 np.array([v for v in V12_cleaned['VEG'].values[nc_loc] if not pd.isnull(v)]).std()

4.3593477839727806

#### MEMUNITS

In [44]:
np.array([v for v in V12_cleaned['MEMUNITS'].values[dementia_loc] if not pd.isnull(v)]).mean()

4.8852798894264

In [45]:
 np.array([v for v in V12_cleaned['MEMUNITS'].values[dementia_loc] if not pd.isnull(v)]).std()

4.227706394212267

#### MMSE

In [46]:
np.array([v for v in V12_cleaned['NACCMMSE'].values[nc_loc] if not pd.isnull(v)]).mean()

28.517761520476657

In [47]:
 np.array([v for v in V12_cleaned['NACCMMSE'].values[nc_loc] if not pd.isnull(v)]).std()

1.7891445793481804

#### ORIENT 

In [48]:
sum([1 for v in V12_cleaned['ORIENT'].values[nc_loc] if v==0])

12626

In [49]:
sum([1 for v in V12_cleaned['ORIENT'].values[nc_loc] if v==0])/len(nc_loc)

0.918989737244341

#### WAIS

In [50]:
np.array([v for v in V12_cleaned['WAIS'].values[dementia_loc] if not pd.isnull(v)]).mean()

33.61023054755043

In [51]:
 np.array([v for v in V12_cleaned['WAIS'].values[dementia_loc] if not pd.isnull(v)]).std()

11.765177662166948

#### LOGIMEM

In [52]:
np.array([v for v in V12_cleaned['LOGIMEM'].values[nc_loc] if not pd.isnull(v)]).mean()

12.546547236257847

In [53]:
 np.array([v for v in V12_cleaned['LOGIMEM'].values[nc_loc] if not pd.isnull(v)]).std()

4.219273167306765

## Missingness 

In [54]:
varmiss=0
for var_ind in range(V12_cleaned.shape[1]):

    variable = V12_cleaned.values[:,var_ind]

        #variable = np.array(variable)
    if np.sum(pd.isnull(variable))>0:
        
        varmiss+=1

print('Number of variables with missing for at least one person: {0}'.format(varmiss))

Number of variables with missing for at least one person: 239


In [55]:
239*100/V12_cleaned.shape[1]

92.63565891472868

In [56]:
personmiss=[]

for row in V12_cleaned.values:
    
    nmiss = sum(np.isnan(row))*100/len(row)
    
    personmiss.append(nmiss)

In [57]:
print('Participant missingness: \n mean {0} \n std {1} \n median {2} \n IQR {3}'.format(np.mean(personmiss), np.std(personmiss), np.median(personmiss), np.percentile(personmiss, [25,75])))

Participant missingness: 
 mean 14.82809455875434 
 std 6.51483554062633 
 median 13.178294573643411 
 IQR [12.01550388 14.34108527]


In [58]:
key_vars = ['MEMORY', 'TRAILB', 'DECCLIN', 'ORIENT', 'HOMEHOBB', 'INDEPEND']

In [59]:
key_data = V12_cleaned[key_vars]

In [60]:
personmiss=[]

for row in key_data.values:
    
    nmiss = sum(np.isnan(row))*100/len(row)
    
    personmiss.append(nmiss)
    
personmiss = np.array(personmiss)

In [61]:
print('Participant missingness: \n mean {0} \n std {1} \n median {2} \n IQR {3}'.format(np.mean(personmiss), np.std(personmiss), np.median(personmiss), np.percentile(personmiss, [25,75])))

Participant missingness: 
 mean 1.132379521351887 
 std 4.207088219711842 
 median 0.0 
 IQR [0. 0.]


In [62]:
len(personmiss[personmiss>0])

1037

In [63]:
varmiss=[]
for var_ind in range(key_data.shape[1]):

    variable = key_data.values[:,var_ind]

        #variable = np.array(variable)
    varmiss.append(np.sum(pd.isnull(variable))/key_data.shape[0])


print('Number of variables with missing for at least one person: {0} {1}'.format(np.mean(varmiss), np.std(varmiss)))

Number of variables with missing for at least one person: 0.011323795213518869 0.02456898250462427


### Impute 

In [64]:
def impute(raw_data):
    
    data = raw_data.copy()

    for var_ind in range(data.shape[1]):

        variable = data[:,var_ind]

        #variable = np.array(variable)
        if np.sum(pd.isnull(variable))>0:

            #print(var_ind)

            sample = variable[~pd.isnull(variable)]
            replace_vals = np.random.choice(sample, np.sum(pd.isnull(variable)), replace=True)
            variable[pd.isnull(variable)] = replace_vals

            data[:,var_ind] = variable
            
    return data

In [65]:
V12_cleaned_imputed = impute(V12_cleaned.values)

### Save data 

In [66]:
import pickle

with open('processed_data.pkl', 'wb') as f:
    
    pickle.dump([V12_cleaned_imputed, V12_cleaned.columns.values, 
                 V12_outcome], f)

In [67]:
np.save('NACCID.npy', V12_used)

## Find misdiagnosed 

#### Misdiganosed are individuals that recieve a dementia diagnosis who revert within 2 years of the diagnosis 

In [68]:
%pdb

Automatic pdb calling has been turned ON


In [69]:
%%time

# misdiagnosed values: -1 = N/A (no dementia diagnosis), 0 = not misdiagnosed, 1 = misdiagnosed
misdiagnosed= []

for i, patient in enumerate(V12_used):
    
    #only patients that are diagnosed with dementia
    
    if V12_outcome[i]==1:
        
        outcomes = data12[(data12['NACCID']==patient) & (data12['NACCVNUM']>1)]
        
        
        #first dementia diagnosis
        
        dem_index = [x for x,v in enumerate(outcomes['NACCUDSD'].values) if v==4]
        no_dem_index = [x for x,v in enumerate(outcomes['NACCUDSD'].values) if v!=4]
        
        loc0 = dem_index[0]
        
        # diagnosis does not change for whole followup
        
        if len(set(outcomes['NACCUDSD'].values[loc0:])) ==1:
            
            misdiagnosed.append(0)
            continue
            
       
        else: #diagnosis changes at some point
            
            #time of first diagnosis

            year0 = outcomes['VISITYR'].values[loc0]
            month0 = outcomes['VISITMO'].values[loc0]
        
            for loc_no in no_dem_index:
                
                if loc_no > loc0:
                    
                    t=loc_no #time of first no dementia diagnosis
                    
                    break

        

            year1 = outcomes['VISITYR'].values[t]
            month1 = outcomes['VISITMO'].values[t]
            
            months = calculate_months(year0, month0, year1, month1) 
            
            if months>29: #diagnosis changed outside 2 year window

                misdiagnosed.append(0)

                continue
                
            else:
                
                misdiagnosed.append(1)
                
                continue

                
    else:
        misdiagnosed.append(-1)
        



CPU times: user 10.2 s, sys: 21.4 ms, total: 10.2 s
Wall time: 10.3 s


In [70]:
sum(1 for m in misdiagnosed if m==1)

130

In [71]:
np.save('misdiagnosed.npy', misdiagnosed)